In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import re
from pathlib import Path
import time

In [2]:
vox_url = 'https://www.vox.com/'
npr_url = 'https://www.npr.org/'

In [3]:
vox_res = requests.get(vox_url)
npr_res = requests.get(npr_url)

print(vox_res.status_code)
print(npr_res.status_code)

200
200


In [4]:
npr_soup = BeautifulSoup(npr_res.content, 'lxml')
vox_soup = BeautifulSoup(vox_res.content, 'lxml')

In [5]:
#whether the article was featured or not
headline = []
date_scraped = []
article_date = []
topic_tag = []
article_text = []
website = []
is_top_story = []

# NPR Scrape

Scraping Featured Stories
- url
- topic tag
- title

In [6]:
#NPR 

#find the section with the headline news and featured articles
group_section = npr_soup.find('div', {'class':'stories-wrap stories-wrap-featured'})

#get only articles
featured_stories = group_section.find_all('div', {'class':'story-wrap'})

npr_urls = []

#append urls for article 
for featured_story in featured_stories: 
    npr_urls.append(featured_story.find('a', 
                                        {'data-metrics': re.compile(
                                            r'{"action" : "Click Story \d+"}'  #any link which brings you to the story
                                        )})['href'])  #grab the url          
    
    
    title = featured_story.find('h3', {'class': 'title'})  # find all titles
#     print(title, '\n')
    headline.append(title.text)                     #append titles to headline list
    
    slug = featured_story.find('h2', {'class':'slug'}) #find section containing topic
    if slug is None:
        topic_tag.append(None)
    else:
        slug = slug.text
#     print(slug.strip(), '\n\n')
        topic_tag.append(slug.strip())                   #append slug/topic to topic list
    
print(len(npr_urls))

7


In [7]:
# print((npr_urls, headline)) 

(['https://www.npr.org/sections/health-shots/2021/10/14/1043414558/with-hospitals-crowded-from-covid-1-in-5-american-families-delays-health-care', 'https://www.npr.org/2021/10/13/1045688076/with-jon-gruden-gone-cheerleaders-and-players-want-the-nfl-to-release-more-data', 'https://www.npr.org/2021/10/13/1045723713/home-heating-costs-this-winter-natural-gas-electric', 'https://www.npr.org/2021/10/08/1044463694/in-squid-game-childrens-games-get-awfully-bloody', 'https://www.npr.org/sections/pictureshow/2021/10/13/1045692906/wildlife-photographer-of-the-year-2021-winners', 'https://www.npr.org/2021/10/14/1045716558/todd-haynes-new-film-takes-us-deep-into-the-velvet-underground', 'https://www.npr.org/2021/10/11/1044973101/cuban-migrant-colombia-darien-gap'], ["1 in 5 American families delayed healthcare due to COVID's strain on hospitals", 'With Jon Gruden gone, cheerleaders and players want the NFL to release more data', 'Winter is coming — and so are high heating bills', "In 'Squid Game,'

In [8]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

7
0
0
7
0
0


Scraping Articles with One Article inside the story

- date of article
- full story
- date of scrape
- website

In [9]:
#setting up counter for creating a bool for the first(top) story
headline_bool = True 

# print(url)
for url in npr_urls:
    page = requests.get(url)
    print(page.status_code)

    article_soup = BeautifulSoup(page.content, 'lxml')

    #website section, including pictures, that holds the p tags (main body of the article)
    text_section = article_soup.select_one('#storytext')
#     print(text_section)

    # some articles' p tags are under a different name than storytext
    if text_section is None:                      #article_soup.select_one(#storytext) ?? article_soup.find(div...)
        text_section = article_soup.find('div', {'class':'ArticlePage-articleBody'})
#     print(text_section)
    
    # Find the images sections of the articles
    images = text_section.find_all('div', {'id': re.compile(r'res\d+')})
    for image in images: 
        image.extract()   #take out images from the text section due to nested 'p' tags

    # Grab the article text and append to the article list 
    full_story = ''
    for p in text_section.select('p'):
        full_story += p.text

    article_text.append(full_story)
#     print(full_story)
    article_date.append(article_soup.find('time')['datetime']) #time of article
    now = datetime.datetime.now() #time of scrape
    
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #code from https://www.programiz.com/python-programming/datetime/current-datetime

    date_scraped.append(dt_string) #append to list 
    
    website.append('NPR')
    print('done')
    
    #True or False if the story is the top_story of the webpage
    is_top_story.append(headline_bool == True)
    headline_bool = False  #increase timer to count for loop runs
    
    time.sleep(4)

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1045823132">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 2657;
        --source-height: 1993;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 763px, (min-width: 1025px) calc(100vw - 496px), (min-width: 768px) calc(100vw - 171px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/13/ap_21259582038866-45527e48cb55ed4336a0164ae42150072756e283-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/13/ap_21259582038866-45527e48cb55ed4336a0164ae42150072756e283-s600-c85.webp 600w,
https://media.npr.org/assets/img/2021/10/13/ap_21259582038866-45527e48cb55ed4336a0164ae42150072756e283-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/13/ap_21259582038866-45527e48cb55ed4336a0164ae42150072756e283-s900-c85.webp 900w,
https://media.npr.org/assets/img/2021/10/13/ap_21259582038866-45527e48

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1045756837">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 2783;
        --source-height: 1853;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 763px, (min-width: 1025px) calc(100vw - 496px), (min-width: 768px) calc(100vw - 171px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/13/ap21286711475326_custom-6d0762c3ae7d92dbdb52fe0e63174352ed3a6d9f-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/13/ap21286711475326_custom-6d0762c3ae7d92dbdb52fe0e63174352ed3a6d9f-s600-c85.webp 600w,
https://media.npr.org/assets/img/2021/10/13/ap21286711475326_custom-6d0762c3ae7d92dbdb52fe0e63174352ed3a6d9f-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/13/ap21286711475326_custom-6d0762c3ae7d92dbdb52fe0e63174352ed3a6d9f-s900-c85.webp 900w,
https://media.npr.org/assets/img/2021/10/13/ap

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1045726454">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 3000;
        --source-height: 1998;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 763px, (min-width: 1025px) calc(100vw - 496px), (min-width: 768px) calc(100vw - 171px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/13/gettyimages-462843546_custom-1fe7be411fb23b647762f95085f2494af4e601a6-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/13/gettyimages-462843546_custom-1fe7be411fb23b647762f95085f2494af4e601a6-s600-c85.webp 600w,
https://media.npr.org/assets/img/2021/10/13/gettyimages-462843546_custom-1fe7be411fb23b647762f95085f2494af4e601a6-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/13/gettyimages-462843546_custom-1fe7be411fb23b647762f95085f2494af4e601a6-s900-c85.webp 900w,
https://media.npr.org/asse

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1044468688">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 4698;
        --source-height: 2642;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 763px, (min-width: 1025px) calc(100vw - 496px), (min-width: 768px) calc(100vw - 171px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/08/squidgame_unit_104_1570_wide-cf094524612a8cfd321074caca0fee236bdd4bee-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/08/squidgame_unit_104_1570_wide-cf094524612a8cfd321074caca0fee236bdd4bee-s600-c85.webp 600w,
https://media.npr.org/assets/img/2021/10/08/squidgame_unit_104_1570_wide-cf094524612a8cfd321074caca0fee236bdd4bee-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/08/squidgame_unit_104_1570_wide-cf094524612a8cfd321074caca0fee236bdd4bee-s900-c85.webp 900w,
https://media.npr.org/asse

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1045743576">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 1913;
        --source-height: 1075;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 763px, (min-width: 1025px) calc(100vw - 496px), (min-width: 768px) calc(100vw - 171px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/13/the_velvet_underground_photo_0101_custom-96b0a15cb2d6a97acd92dae1becf76ef426f07a2-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/13/the_velvet_underground_photo_0101_custom-96b0a15cb2d6a97acd92dae1becf76ef426f07a2-s600-c85.webp 600w,
https://media.npr.org/assets/img/2021/10/13/the_velvet_underground_photo_0101_custom-96b0a15cb2d6a97acd92dae1becf76ef426f07a2-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/13/the_velvet_underground_photo_0101_custom-96b0a15cb2d6a97acd92dae1becf76ef426f07a

200
<div class="storytext storylocation linkLocation" id="storytext">
<div class="bucketwrap image large" id="res1045631392">
<div class="imagewrap has-source-dimensions" data-crop-type="" style="
        --source-width: 2997;
        --source-height: 1998;
    ">
<picture>
<source class="img" sizes="(min-width: 1300px) 1238px, (min-width: 1025px) calc(100vw - 60px), (min-width: 768px) calc(100vw - 60px), calc(100vw - 30px)" srcset="https://media.npr.org/assets/img/2021/10/13/darien-gap-migration0291_slide-b3a9f2117778454556e7504a2b8c48962d632d89-s400-c85.webp 400w,
https://media.npr.org/assets/img/2021/10/13/darien-gap-migration0291_slide-b3a9f2117778454556e7504a2b8c48962d632d89-s800-c85.webp 800w,
https://media.npr.org/assets/img/2021/10/13/darien-gap-migration0291_slide-b3a9f2117778454556e7504a2b8c48962d632d89-s1000-c85.webp 1000w,
https://media.npr.org/assets/img/2021/10/13/darien-gap-migration0291_slide-b3a9f2117778454556e7504a2b8c48962d632d89-s1300-c85.webp 1300w,
https://media.n

In [10]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

7
7
7
7
7
7


# Vox 

Scraping:
- Title

In [11]:
#find the section with the headline news and featured articles
group_section = vox_soup.find('div', {'class':'c-newspaper__main'})

#find the tag containing the titles for each featured article
titles = group_section.find_all('a', {'data-chorus-optimize-field':'hed'}) 

vox_urls = []
for title in titles:
    print(title.text)
    headline.append(title.text)  #append title to the headline list
    vox_urls.append(title['href']) #append url to a list 

Can the US cut drug prices without sacrificing new cures?
The dark, enthralling power of Succession
Bryan Stevenson on tracing the legacy of American enslavement to mass incarceration
Biden’s incoherent immigration policy
Why Hollywood loves this creepy bird call
When removing a statue actually is antidemocratic
Maybe losing the AI race to China isn’t such a bad idea
Why so many people undercharge for their work


In [12]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

15
7
7
7
7
7


- article_text
- date_scraped
- website
- topic_tag
- article_date
- website

In [13]:
headline_bool = True

for url in vox_urls:
    page = requests.get(url)
    print(page.status_code)
    article_soup = BeautifulSoup(page.content, 'lxml')

    
    text_section = article_soup.find('div', {'class':'c-entry-content'}) #isolate the text only from the article
    story_text = text_section.find_all('p') #grab all paragraph tags
    
    full_story = ''   #make an empty string to append each p to
    
    #get text under p tag and add to the full article story
    for p in story_text:
        full_story+= p.text
        
    article_text.append(full_story) #add to list of articles
    
    article_date.append(article_soup.find('time')['datetime']) #time of article
    now = datetime.datetime.now() #get current time of scrape
    
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    date_scraped.append(dt_string)
    #code from https://www.programiz.com/python-programming/datetime/current-datetime    
    website.append('Vox')  #record website of article origin
    
    topic_tag.append(None)
    
    is_top_story.append(headline_bool == True)
    headline_bool = False


200
200
200
200
200
200
200
200


In [14]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

15
15
15
15
15
15


# Create DataFrame 

In [15]:
vox_npr_df = pd.DataFrame({'headline':headline,
                              'date_scraped':date_scraped,
                              'article_date':article_date,
                              'topic_tag':topic_tag,
                              'article_text':article_text,
                              'website':website})

In [16]:
#####If creating new csv per scrape uncomment code##########

# filename = 'scrapes' + datetime.datetime.now().strftime("%d-%m-%Y_%H:%M")

# vox_npr_df.to_csv(filename+'.csv', index = False)

# Saving/Appending to CSV

In [17]:
file_name = Path("scraped_lib_articles.csv") #may need to change to os if we use s3 buckets

if file_name.exists():  #if file already exists in directory
    print('file exists, now appending')
    
    #append new scrapes to the file without header row
    vox_npr_df.to_csv('scraped_lib_articles.csv', mode='a', index=False, header=False) 

else: #if file does not exist yet
    print('file does not exist, now creating')
    vox_npr_df.to_csv('scraped_lib_articles.csv', index=False) #create new file 

file exists, now appending
